In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'TINYAES128C'
VERSION = 'HARDWARE'
SS_VER = 'SS_VER_1_1'

In [2]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:701) Your firmware (0.24) is outdated - latest is 0.62See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [3]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
C:/Users/yitsewu/CHIPWH~1/cw/home/portable/avrgcc/bin/make clean_objs .dep 
make[1]: Entering directory 'C:/Users/yitsewu/ChipWhisperer5_64/cw/home/portable/chipwhisperer/hardware/victims/firmware/simpleserial-aes'
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
rm -f -- simpleserial-aes-CWNANO.hex
rm -f -- simpleserial-aes-CWNANO.eep
rm -f -- simpleserial-aes-CWNANO.cof
rm -f -- simpleserial-aes-CWNANO.elf
rm -f -- simpleserial-aes-CWNANO.map
rm -f -- simpleserial-aes-CWNANO.sym
rm -f -- simpleserial-aes-CWNANO.lss
rm -f -- objdir-CWNANO/*.o
rm -f -- objdir-CWNANO/*.lst
rm -f -- simpleserial-aes.s simpleserial.s stm32f0_hal_nano.s stm32f0_hal_lowlevel.s aes.s aes-independant.s
rm -f -- simpleserial-aes.d simpleserial.d stm32f0_hal_nano.d stm32f0_hal_lowlevel.d aes.d aes-independant.

In [4]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5971 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5971 bytes


In [5]:
#Capture Traces
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
trace_num = 300  # Number of traces

proj_name = f"traces/CWNANO_{CRYPTO_TARGET}_{trace_num}_fixedkey.cwp"
project = cw.create_project(proj_name, overwrite=True)

for i in trange(trace_num, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

print(f'Number of samples = {scope.adc.samples}')
project.save()

Capturing traces:   0%|          | 0/300 [00:00<?, ?it/s]

Number of samples = 5000


In [6]:
scope.dis()
target.dis()

In [7]:
import chipwhisperer.analyzer as cwa
#pick right leakage model for your attack
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)
results = attack.run(cwa.get_jupyter_callback(attack))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2B0.525,7E0.605,150.558,160.524,280.514,AE0.561,D20.481,A60.604,AB0.584,F70.546,150.522,880.590,090.503,CF0.606,4F0.522,3C0.548
1,C00.305,090.311,2A0.282,6B0.278,670.284,FE0.289,640.271,570.327,070.286,D10.293,D00.284,840.277,1E0.262,160.279,270.287,820.270
2,C80.262,2D0.274,D60.268,580.268,740.276,940.278,910.263,790.271,220.268,CE0.284,5C0.277,860.269,EC0.260,E40.269,CB0.261,E80.267
3,5B0.262,7D0.268,1E0.264,AD0.260,A30.264,D90.269,DB0.263,BD0.271,250.268,150.275,160.272,8E0.268,690.259,260.262,720.260,030.264
4,CA0.259,AF0.263,B40.259,670.255,0A0.259,CF0.266,590.259,B60.266,680.263,FA0.272,520.261,8A0.267,120.255,880.262,A60.255,910.264


In [8]:
# snr = cwa.calculate_snr(project.traces, leak_model, bnum=1, db=False)
# cw.plot(snr)

In [9]:
print(project.keys[0])

[ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]


In [10]:
# cw.plot(project.waves[0])

In [11]:
# The AES SBox that we will use to generate our labels
AES_Sbox = np.array([
            0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
            0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
            0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
            0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
            0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
            0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
            0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
            0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
            0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
            0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
            0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
            0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
            0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
            0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
            0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
            0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
            ])
# The Hamming weight list of each byte value
hw_list = np.array([
            0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 
            1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 
            1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 
            2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 
            1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6,
            2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7,
            2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7,
            3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 4, 5, 5, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8
            ])
# Number of combinations for each Hamming weight
hw_combinations = np.array([1, 8, 28, 56, 70, 56, 28, 8, 1])

In [12]:
t = np.array(project.waves)
p = np.array(project.textins)
k = np.array(project.keys)
# c = np.array(proj.textouts)

print(t.shape)
(trace_num, trace_len) = t.shape
print(f'{trace_num} trace, each with {trace_len} points.')

(300, 5000)
300 trace, each with 5000 points.


In [13]:
byte_num = 16
key_num = 256

In [14]:
print(hw_list[AES_Sbox[p[0, 0] ^ np.arange(key_num)]])
print(t[0:10])

[4 3 4 6 4 5 4 6 4 4 3 4 3 4 5 4 4 5 3 3 5 5 3 4 6 3 2 2 5 4 3 5 2 7 5 4 4
 5 5 4 4 3 4 5 3 2 4 5 6 3 3 3 4 4 2 4 2 3 7 5 3 3 3 3 3 5 3 5 6 5 3 5 2 4
 6 5 3 5 5 2 3 6 6 5 4 2 3 6 2 2 2 3 5 5 2 2 4 5 3 4 4 3 3 5 2 4 3 2 4 2 3
 3 5 1 4 5 5 5 4 4 4 4 4 5 5 5 6 3 5 2 3 5 5 5 3 5 4 2 4 5 3 3 2 3 3 6 3 3
 6 4 4 5 4 5 1 6 0 6 4 4 6 3 2 4 1 7 3 6 4 3 3 4 4 7 3 7 6 4 1 8 5 2 5 5 3
 6 3 5 1 5 3 4 5 5 7 6 5 4 2 1 6 4 5 5 6 6 4 5 4 2 4 3 3 6 5 4 4 4 6 4 4 6
 4 2 3 3 4 4 5 5 3 4 7 4 4 6 4 3 6 7 4 5 6 4 1 4 3 2 2 4 2 4 3 4 1 5]
[[-0.15234375 -0.08203125 -0.1875     ... -0.15234375 -0.13671875
  -0.08984375]
 [-0.40625    -0.078125   -0.18359375 ... -0.1484375  -0.13671875
  -0.08984375]
 [-0.4140625  -0.08203125 -0.1875     ... -0.14453125 -0.13671875
  -0.07421875]
 ...
 [-0.40234375 -0.0859375  -0.18359375 ... -0.140625   -0.12890625
  -0.078125  ]
 [-0.41015625 -0.078125   -0.17578125 ... -0.15234375 -0.13671875
  -0.07421875]
 [-0.40234375 -0.078125   -0.18359375 ... -0.15625    -0.13671875
  -0

In [15]:
h_sum = np.zeros((byte_num, key_num))
h2_sum = np.zeros((byte_num, key_num))
t_sum = np.zeros((1, trace_len))
t2_sum = np.zeros((1, trace_len))
h_t_sum = np.zeros((byte_num, key_num, trace_len))
r = np.zeros((byte_num, key_num, trace_len))
key_guess = np.zeros(byte_num)

for i in range(trace_num):
    n = i+1
    t_sum += t[i]
    t2_sum += t[i]**2
    std_t = np.sqrt(t2_sum - t_sum**2 / n)
    
    for b in range(byte_num):
        h = hw_list[AES_Sbox[p[i, b] ^ np.arange(key_num)]]
        h_sum[b] += h
        h2_sum[b] += h**2
        h_t_sum[b] += np.outer(h, t[i])
        
        nom = h_t_sum[b] - np.outer(h_sum[b], t_sum) / n
        denom = np.outer(np.sqrt(h2_sum[b] - h_sum[b]**2 / n), std_t)
        r[b] = nom / denom

        index = np.unravel_index(np.argmax(np.absolute(r[b]), axis=None), r[b].shape)
        
        key_guess[b] = index[0]
        
    print(key_guess)

C:\Users\yitsewu\CHIPWH~1\cw\home\portable\WPy64-3771\python-3.7.7.amd64\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43

[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43. 126.  21.  22.  40. 174. 210. 166. 171. 247.  21. 136.   9. 207.
  79.  60.]
[ 43

In [16]:
print(k[0])

[ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]
